#Project Explanation
##subheading

In [1]:
#importing all libraries that I will need for this project
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [14]:
#internet file - WHO Data Density of physicians (per 10 000 population) [CSV]
#reading from internet using read_csv function
density = pd.read_csv('https://srhdpeuwpubsa.blob.core.windows.net/whdh/DATADOT/INDICATOR/217795A_ALL_LATEST.csv')
density.head()

#data exploration
print('rows, cols', density.shape)
print(density.head())
print(density.describe())

#transforming data ~
#subsetting to only certain years to align with other datasets
print(density["DIM_TIME"].dtype)#checking datatype of year column
density = density[density["DIM_TIME"].isin([2015,2016,2017])]

#removing unnecessary columns and renaming the columns
print(density.columns)
density_clean = density[["DIM_TIME","GEO_NAME_SHORT","RATE_PER_10000_N"]]
density_clean.rename(columns={'DIM_TIME':'Year', 'GEO_NAME_SHORT':'Country', 'RATE_PER_10000_N':'Density'}, inplace=True)
density_clean.head()

rows, cols (3168, 12)
            IND_ID  IND_CODE IND_UUID IND_PER_CODE  DIM_TIME DIM_TIME_TYPE  \
0  217795AHWF_0001  HWF_0001  217795A     HWF_0001      2006          YEAR   
1  217795AHWF_0001  HWF_0001  217795A     HWF_0001      2012          YEAR   
2  217795AHWF_0001  HWF_0001  217795A     HWF_0001      2020          YEAR   
3  217795AHWF_0001  HWF_0001  217795A     HWF_0001      2016          YEAR   
4  217795AHWF_0001  HWF_0001  217795A     HWF_0001      2018          YEAR   

   DIM_GEO_CODE_M49 DIM_GEO_CODE_TYPE DIM_PUBLISH_STATE_CODE  \
0               826           COUNTRY              PUBLISHED   
1               558           COUNTRY              PUBLISHED   
2               608           COUNTRY              PUBLISHED   
3               620           COUNTRY              PUBLISHED   
4               620           COUNTRY              PUBLISHED   

             IND_NAME                                     GEO_NAME_SHORT  \
0  Density of doctors  United Kingdom of Great B

<ipython-input-14-8edef7be6326>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  density_clean.rename(columns={'DIM_TIME':'Year', 'GEO_NAME_SHORT':'Country', 'RATE_PER_10000_N':'Density'}, inplace=True)


,Year,Country,Density
3,2016,Portugal,49.362
10,2016,Comoros,2.895
13,2015,France,32.391
18,2015,Latvia,31.747
20,2017,Saudi Arabia,24.093


In [16]:
#local file - World Bank Group World Development Indicators [Tab-Delimited Text File]
wdi = pd.read_csv('/content/7f00b99d-25a1-442f-88e1-966fddb29125_Data.txt', sep='\t')
#data exploration
print('rows, cols', wdi.shape)
print(wdi.head())
print(wdi.describe())
#data transformation - renaming the year columns to mention just the year and subsetting data
wdi.rename(columns={'2015 [YR2015]':'2015', '2016 [YR2016]':'2016', '2017 [YR2017]':'2017'}, inplace=True)
wdi = wdi[["Country Name","Series Name", "2015", "2016", "2017"]]
wdi.head()
#data pivoting (unpivoting the year columns - wide to long)
wdi_clean = wdi.melt(id_vars=['Country Name', 'Series Name'], value_vars=['2015', '2016', '2017'], var_name='Year', value_name='Value')
wdi_clean.head()

rows, cols (537, 7)
                                         Series Name     Series Code  \
0  Births attended by skilled health staff (% of ...  SH.STA.BRTC.ZS   
1  Births attended by skilled health staff (% of ...  SH.STA.BRTC.ZS   
2  Births attended by skilled health staff (% of ...  SH.STA.BRTC.ZS   
3  Births attended by skilled health staff (% of ...  SH.STA.BRTC.ZS   
4  Births attended by skilled health staff (% of ...  SH.STA.BRTC.ZS   

     Country Name Country Code 2015 [YR2015] 2016 [YR2016] 2017 [YR2017]  
0     Afghanistan          AFG          50.5            ..          53.4  
1         Albania          ALB            ..            ..            ..  
2         Algeria          DZA            ..            ..            ..  
3  American Samoa          ASM            ..            ..            ..  
4         Andorra          AND           100           100           100  
                                              Series Name     Series Code  \
count               

,Country Name,Series Name,Year,Value
0,Afghanistan,Births attended by skilled health staff (% of ...,2015,50.5
1,Albania,Births attended by skilled health staff (% of ...,2015,..
2,Algeria,Births attended by skilled health staff (% of ...,2015,..
3,American Samoa,Births attended by skilled health staff (% of ...,2015,..
4,Andorra,Births attended by skilled health staff (% of ...,2015,100


In [4]:
#local file - OECD Healthcare Coverage Data - Selected Dates [CSV]
oecd = pd.read_csv('/content/OECD.ELS.HD,DSD_HEALTH_PROT@DF_HEALTH_PROT,1.0+.A..PT_POP.COVGCMED+TPRIBASI.csv')
#data exploration
print('rows, cols', oecd.shape)
print(oecd.head())
print(oecd.describe())



rows, cols (232, 24)
  STRUCTURE                                     STRUCTURE_ID  \
0  DATAFLOW  OECD.ELS.HD:DSD_HEALTH_PROT@DF_HEALTH_PROT(1.0)   
1  DATAFLOW  OECD.ELS.HD:DSD_HEALTH_PROT@DF_HEALTH_PROT(1.0)   
2  DATAFLOW  OECD.ELS.HD:DSD_HEALTH_PROT@DF_HEALTH_PROT(1.0)   
3  DATAFLOW  OECD.ELS.HD:DSD_HEALTH_PROT@DF_HEALTH_PROT(1.0)   
4  DATAFLOW  OECD.ELS.HD:DSD_HEALTH_PROT@DF_HEALTH_PROT(1.0)   

        STRUCTURE_NAME ACTION REF_AREA Reference area FREQ  \
0  Healthcare coverage      I      NLD    Netherlands    A   
1  Healthcare coverage      I      NLD    Netherlands    A   
2  Healthcare coverage      I      NLD    Netherlands    A   
3  Healthcare coverage      I      EST        Estonia    A   
4  Healthcare coverage      I      EST        Estonia    A   

  Frequency of observation MEASURE                    Measure  ...  \
0                   Annual     HIC  Health insurance coverage  ...   
1                   Annual     HIC  Health insurance coverage  ...   
2          